# Most Dangerous Streets - Month Comparison

Compare the most recent month to the all-time numbers to see which streets got safer. 

In [1]:
import pytz
import folium
import pandas as pd
from datetime import datetime, timedelta
from crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()

In [3]:
start_date = pytz.timezone('America/Denver').localize(datetime(2013,1,1))
end_date = cda.most_recent_crash_timestamp()

all_time = cda.street_metrics(start_date, end_date)

In [4]:
start_date = pytz.timezone('America/Denver').localize(datetime(2023,1,1))
end_date = pytz.timezone('America/Denver').localize(datetime(2023,2,1))

last_month = cda.street_metrics(start_date, end_date)

In [5]:
# A street must have at least 100 crashes all-time to be in the comparison
all_time = all_time[all_time.num_crashes >= 100].copy()

In [6]:
comparison = pd.merge(
    all_time
    , last_month
    , how='inner'
    , on=['gid', 'lrsroute', 'fullname', 'length_miles','street_line']
    , suffixes=['_all_time', '_last_month']
)

In [7]:
comparison['diff_pmpw'] = (
    comparison['crashes_per_mile_per_week_last_month']
    - comparison['crashes_per_mile_per_week_all_time']
)

In [8]:
comparison['expected_crashes'] = (
    comparison['length_miles']
    * comparison['crashes_per_mile_per_week_all_time']
    * (comparison['days_in_data_last_month'] / 7)
)

comparison['diff_expected_crashes'] = comparison['num_crashes_last_month'] - comparison['expected_crashes']

## Difference in Crash Rate per Mile

In [9]:
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Toner')

good_streets = comparison.sort_values(by='diff_pmpw').head(5)['street_line']
bad_streets = comparison.sort_values(by='diff_pmpw').tail(5)['street_line']

for good_street in good_streets:
    folium.GeoJson(good_street, style_function=lambda x: {'color': 'green'}).add_to(this_map)

for bad_street in bad_streets:
    folium.GeoJson(bad_street, style_function=lambda x: {'color': 'red'}).add_to(this_map)
    
this_map.fit_bounds(this_map.get_bounds())
this_map

In [10]:
comparison.sort_values(by='diff_pmpw')[[
    'gid'
    , 'fullname'
    , 'length_miles'
    , 'crashes_per_mile_per_week_all_time'
    , 'crashes_per_mile_per_week_last_month'
    , 'diff_pmpw'
    , 'num_crashes_all_time'
    , 'num_crashes_last_month'
    , 'expected_crashes'
    , 'diff_expected_crashes'
]]

,gid,fullname,length_miles,crashes_per_mile_per_week_all_time,crashes_per_mile_per_week_last_month,diff_pmpw,num_crashes_all_time,num_crashes_last_month,expected_crashes,diff_expected_crashes
31,5125,N KALAMATH ST,1.755956,0.947928,0.257189,-0.690739,886,2,7.371444,-5.371444
4,7584,S FEDERAL BLVD,4.373528,1.555864,0.877715,-0.678150,3622,17,30.134729,-13.134729
37,967,AURARIA PKWY,0.873770,0.911641,0.258428,-0.653213,424,1,3.527644,-2.527644
15,2127,N PEORIA ST,2.089566,1.287485,0.648383,-0.639102,1432,6,11.914117,-5.914117
26,4256,38TH ST,0.518240,1.047663,0.435718,-0.611946,289,1,2.404455,-1.404455
...,...,...,...,...,...,...,...,...,...,...
36,1453,E SPEER BLVD,0.976285,0.917904,1.850333,0.932429,477,8,3.968599,4.031401
106,4236,27TH ST,0.589821,0.541481,1.531355,0.989873,170,4,1.414385,2.585615
48,7725,S LINCOLN ST,1.069023,0.817187,1.901043,1.083856,465,9,3.868760,5.131240
71,577,E 56TH AVE,0.727734,0.676369,2.172009,1.495640,262,7,2.179817,4.820183


## Difference in Number of Expected Crashes

In [11]:
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Toner')

good_streets = comparison.sort_values(by='diff_expected_crashes').head(5)['street_line']
bad_streets = comparison.sort_values(by='diff_expected_crashes').tail(5)['street_line']

for good_street in good_streets:
    folium.GeoJson(good_street, style_function=lambda x: {'color': 'green'}).add_to(this_map)

for bad_street in bad_streets:
    folium.GeoJson(bad_street, style_function=lambda x: {'color': 'red'}).add_to(this_map)
    
this_map.fit_bounds(this_map.get_bounds())
this_map

In [12]:
comparison.sort_values(by='diff_expected_crashes')[[
    'gid'
    , 'fullname'
    , 'length_miles'
    , 'crashes_per_mile_per_week_all_time'
    , 'crashes_per_mile_per_week_last_month'
    , 'diff_pmpw'
    , 'expected_crashes'
    , 'num_crashes_last_month'
    , 'diff_expected_crashes'
]]

,gid,fullname,length_miles,crashes_per_mile_per_week_all_time,crashes_per_mile_per_week_last_month,diff_pmpw,expected_crashes,num_crashes_last_month,diff_expected_crashes
4,7584,S FEDERAL BLVD,4.373528,1.555864,0.877715,-0.678150,30.134729,17,-13.134729
8,1012,E COLFAX AVE,5.475202,1.446624,1.072283,-0.374341,35.076758,26,-9.076758
10,1016,N COLORADO BLVD,5.184503,1.383878,1.001745,-0.382133,31.773752,23,-8.773752
18,4689,N FEDERAL BLVD,5.367276,1.153688,0.799348,-0.354340,27.422437,19,-8.422437
74,6481,S SHERIDAN BLVD,5.380667,0.658508,0.335730,-0.322778,15.691358,8,-7.691358
...,...,...,...,...,...,...,...,...,...
71,577,E 56TH AVE,0.727734,0.676369,2.172009,1.495640,2.179817,7,4.820183
166,1430,GREEN VALLEY RANCH BLVD,4.075550,0.397353,0.664862,0.267509,7.171766,12,4.828234
189,6759,E YALE AVE,2.522337,0.361983,0.805704,0.443721,4.043478,9,4.956522
48,7725,S LINCOLN ST,1.069023,0.817187,1.901043,1.083856,3.868760,9,5.131240


In [13]:
comparison = comparison.sort_values(by='diff_expected_crashes')

comparison.loc[comparison['diff_expected_crashes'].rank() <= 5, 'leaderboard'] = (
    comparison.loc[comparison['diff_expected_crashes'].rank() <= 5].apply(
        lambda row: f'{row.diff_expected_crashes:.0f} crashes'
        , axis=1
    )
)

comparison.loc[comparison['diff_expected_crashes'].rank(ascending=False) <= 5, 'leaderboard'] = (
    comparison.loc[comparison['diff_expected_crashes'].rank(ascending=False) <= 5].apply(
        lambda row: f'+{row.diff_expected_crashes:.0f} crashes'
        , axis=1
    )
)


In [14]:
comparison['street_name'] = comparison['fullname'].str.title()
comparison.loc[comparison.leaderboard.notnull(), ['street_name', 'leaderboard']]

,street_name,leaderboard
4,S Federal Blvd,-13 crashes
8,E Colfax Ave,-9 crashes
10,N Colorado Blvd,-9 crashes
18,N Federal Blvd,-8 crashes
74,S Sheridan Blvd,-8 crashes
71,E 56Th Ave,+5 crashes
166,Green Valley Ranch Blvd,+5 crashes
189,E Yale Ave,+5 crashes
48,S Lincoln St,+5 crashes
211,N Tower Rd,+8 crashes
